# Locales y Terrazas Comunidad de Madrid

Objetivo: identificar restaurantes y bares de la Comunidad de Madrid que están activos y si tienen terraza.

Hacer las transformaciones necesarias y guardar los datos obtenidos en fichero formato json.

Importar librerias necesarias y crear la sesion de spark

In [28]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import functions as pyf

import utm
import pandas as pd 
import numpy as np 
import json

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

#Crear la sesión de spark
spark = SparkSession.builder.appName('datos_proyecto').getOrCreate()

Leer los ficheros csv que vamos a utilizar

In [4]:
#Ficheros
fichero1 = r'C:\Users\usuario\Documents\Blanca\Spark\3.Practica_proyecto\Practica_proyecto\Datos\OPEN DATA Locales-Epigrafes202010 (1).csv'
fichero2 = r'C:\Users\usuario\Documents\Blanca\Spark\3.Practica_proyecto\Practica_proyecto\Datos\OPEN DATA Terrazas20209.csv'

In [5]:
#Utilizando () para envolver el código, no es necesario añadir "\" para saltar de linea.
locales = (spark.read.csv(fichero1, header=True, inferSchema=True, sep=';')
        .select('rotulo', 'id_local', 'clase_vial_acceso', 'desc_vial_acceso', 'num_edificio', 
        'desc_situacion_local', 'desc_seccion', 'desc_epigrafe', 'desc_distrito_local', 'coordenada_x_local','coordenada_y_local') 
        .filter((pyf.col('desc_seccion') == 'HOSTELERIA') & (pyf.col('desc_situacion_local') == 'Abierto'))
)
#Directamente al leer el csv seleccionamos las columnas que nos inetersan, la actividad y la situación del local  

In [6]:
terrazas = (spark.read.csv(fichero2, header=True, inferSchema=True, sep=';')
        .select('id_terraza', 'id_local', 'desc_situacion_terraza')
        .filter(pyf.col('desc_situacion_terraza') == 'Abierta')
)

In [7]:
#Vista global del fichero de locales
locales.show(2)
locales.printSchema()
print(locales.count())

+--------------------+---------+--------------------+--------------------+------------+--------------------+------------+---------------+--------------------+------------------+------------------+
|              rotulo| id_local|   clase_vial_acceso|    desc_vial_acceso|num_edificio|desc_situacion_local|desc_seccion|  desc_epigrafe| desc_distrito_local|coordenada_x_local|coordenada_y_local|
+--------------------+---------+--------------------+--------------------+------------+--------------------+------------+---------------+--------------------+------------------+------------------+
|LA NI�ATA CERVECE...|285016182|RONDA            ...|CABALLERO DE LA M...|          69|             Abierto|  HOSTELERIA| BAR CON COCINA|FUENCARRAL-EL PARDO |         441353,63|        4483636,55|
|           PEGGY SUE|285016194|AVENIDA          ...|MONFORTE DE LEMOS...|         185|             Abierto|  HOSTELERIA|BAR RESTAURANTE|FUENCARRAL-EL PARDO |         439136,63|        4480869,56|
+--------------

In [8]:
#Vista global del fochero terrazas
terrazas.show(2)
terrazas.printSchema()
print(terrazas.count())

+----------+---------+----------------------+
|id_terraza| id_local|desc_situacion_terraza|
+----------+---------+----------------------+
|      1283|285029169|               Abierta|
|      1284|200001006|               Abierta|
+----------+---------+----------------------+
only showing top 2 rows

root
 |-- id_terraza: integer (nullable = true)
 |-- id_local: integer (nullable = true)
 |-- desc_situacion_terraza: string (nullable = true)

5714


In [9]:
#Selección solo de algunos epígrafes del primer csv
locales_filtro = locales.filter((pyf.col('desc_epigrafe') == 'CAFETERIA') | (pyf.col('desc_epigrafe') == "BAR SIN COCINA") | (pyf.col('desc_epigrafe') =='BAR CON COCINA') | (pyf.col('desc_epigrafe') =='RESTAURANTE') | (pyf.col('desc_epigrafe') =='BAR RESTAURANTE') | (pyf.col('desc_epigrafe') =='BAR ESPECIAL SIN ACTUACIONES') | (pyf.col('desc_epigrafe') =='RESTAURANTES DE COMIDA RAPIDA') | (pyf.col('desc_epigrafe') =='TABERNA') | (pyf.col('desc_epigrafe') =='HOTELES Y MOTELES CON RESTAURANTE') | (pyf.col('desc_epigrafe') =='BAR ESPECIAL CON ACTUACIONES') | (pyf.col('desc_epigrafe') =='CHOCOLATERIA/SALON DE TE Y HELADERIA'))

In [10]:
#Covertir tipo de algunas columnas
locales_filtro = locales_filtro.withColumn('num_edificio', locales_filtro['num_edificio'].cast('string')) \
        .withColumn('id_local', locales_filtro['id_local'].cast('integer'))

In [11]:
#Cambiar el formato de las coordenadas para luego transformarlas a coordenadas utm
#1.Cambiar "," por "."
locales_filtro = locales_filtro.withColumn('coordenada_x_local', pyf.regexp_replace('coordenada_x_local', ',', '.')) \
                .withColumn('coordenada_y_local', pyf.regexp_replace('coordenada_y_local', ',', '.'))

In [12]:
#2.String to float
locales_filtro = locales_filtro.withColumn('coordenada_x_local', locales_filtro['coordenada_x_local'].cast('float')) \
                .withColumn('coordenada_y_local', locales_filtro['coordenada_y_local'].cast('float')) \

In [13]:
#Agrupación de la dirección del local en una sola columna
locales_filtro = locales_filtro.withColumn('direccion', pyf.concat(pyf.col('clase_vial_acceso'), pyf.lit(" "), pyf.col('desc_vial_acceso'), pyf.lit(" "), pyf.col('num_edificio')))

In [14]:
#Total locales disponibles
print(locales_filtro.count())

18072


In [15]:
#Unir la tabla locales con terrazas JOIN
locales_total = locales_filtro.join(terrazas, on=['id_local'], how='left_outer')
locales_total.show(2)
print(locales_total.count())

+---------+--------------------+--------------------+--------------------+------------+--------------------+------------+---------------+--------------------+------------------+------------------+--------------------+----------+----------------------+
| id_local|              rotulo|   clase_vial_acceso|    desc_vial_acceso|num_edificio|desc_situacion_local|desc_seccion|  desc_epigrafe| desc_distrito_local|coordenada_x_local|coordenada_y_local|           direccion|id_terraza|desc_situacion_terraza|
+---------+--------------------+--------------------+--------------------+------------+--------------------+------------+---------------+--------------------+------------------+------------------+--------------------+----------+----------------------+
|285016182|LA NI�ATA CERVECE...|RONDA            ...|CABALLERO DE LA M...|          69|             Abierto|  HOSTELERIA| BAR CON COCINA|FUENCARRAL-EL PARDO |         441353.62|         4483636.5|RONDA            ...|      7367|               A

In [16]:
#Añadir una nueva columna para identificar los locales con terreza (si=1, no=0)
locales_total = locales_total.withColumn('terraza', pyf.when(pyf.col('id_terraza') >0, 1).otherwise(0))
locales_total.show(3)

+---------+--------------------+--------------------+--------------------+------------+--------------------+------------+---------------+--------------------+------------------+------------------+--------------------+----------+----------------------+-------+
| id_local|              rotulo|   clase_vial_acceso|    desc_vial_acceso|num_edificio|desc_situacion_local|desc_seccion|  desc_epigrafe| desc_distrito_local|coordenada_x_local|coordenada_y_local|           direccion|id_terraza|desc_situacion_terraza|terraza|
+---------+--------------------+--------------------+--------------------+------------+--------------------+------------+---------------+--------------------+------------------+------------------+--------------------+----------+----------------------+-------+
|285016182|LA NI�ATA CERVECE...|RONDA            ...|CABALLERO DE LA M...|          69|             Abierto|  HOSTELERIA| BAR CON COCINA|FUENCARRAL-EL PARDO |         441353.62|         4483636.5|RONDA            ...|   

In [17]:
print(locales_total.count())

18072


Tratamiento de las coordenadas

In [18]:
#Seleccionamos solo las filas que tienen coordenadas coorectas
locales_total = locales_total.filter(pyf.col('coordenada_x_local') > 400000)

In [19]:
print(locales_total.count())

17084


In [20]:
#Covertir a coordenadas utm
utm_x = pyf.udf(lambda x, y: float(utm.to_latlon(x,y, 30, 'T')[0]), FloatType())
utm_y = pyf.udf(lambda x, y: float(utm.to_latlon(x,y, 30, 'T')[1]), FloatType())
    

In [21]:
locales_total = locales_total.withColumn('lat', utm_x(pyf.col('coordenada_x_local'), pyf.col('coordenada_y_local')))
locales_total = locales_total.withColumn('lon', utm_y(pyf.col('coordenada_x_local'), pyf.col('coordenada_y_local')))

## Fichero definitivo

In [32]:
locales_total = locales_total.select('id_local', 'desc_distrito_local', 'rotulo', 'direccion', 'terraza', 'lat', 'lon')
locales_total.show(4)

+---------+--------------------+--------------------+--------------------+-------+---------+----------+
| id_local| desc_distrito_local|              rotulo|           direccion|terraza|      lat|       lon|
+---------+--------------------+--------------------+--------------------+-------+---------+----------+
|285016182|FUENCARRAL-EL PARDO |LA NI�ATA CERVECE...|RONDA            ...|      1|40.501366|-3.6921358|
|285016194|FUENCARRAL-EL PARDO |           PEGGY SUE|AVENIDA          ...|      1| 40.47628|-3.7180328|
|285016368|FUENCARRAL-EL PARDO |      LA TAGLIATELLA|PASEO            ...|      0|40.504257|-3.6707327|
|285016372|FUENCARRAL-EL PARDO |  MANOLITO POR DIOS!|PASEO            ...|      0|40.504517|-3.6710894|
+---------+--------------------+--------------------+--------------------+-------+---------+----------+
only showing top 4 rows



In [33]:
#Convertir a un datframe de pandas
locales_df = locales_total.toPandas()

In [35]:
#Saving the pandas df to json file
j = locales_df.to_json(orient='records')
with open('locales_prueba.json', 'w') as json_file:
    json.dump(j, json_file)